In [3]:
# JJ Rolling Pivots

import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [215]:
bnf_1hr = pd.read_csv("../data/crude_4hr_tv.csv")
bnf_1hr["datetime"] = pd.to_datetime(bnf_1hr["time"])
bnf_1hr = bnf_1hr.drop(columns=["time"])
bnf_1hr = bnf_1hr[
    (bnf_1hr["datetime"].dt.year >= 2011) & (bnf_1hr["datetime"].dt.year <= 2024)
]
bnf_1hr.tail()

,open,high,low,close,datetime
9309,6746,6753,6716,6730,2024-06-19 21:00:00+05:30
9310,6729,6743,6719,6738,2024-06-20 09:00:00+05:30
9311,6740,6766,6719,6759,2024-06-20 13:00:00+05:30
9312,6760,6818,6750,6772,2024-06-20 17:00:00+05:30
9313,6772,6786,6768,6777,2024-06-20 21:00:00+05:30


In [216]:
PORTFOLIO = 50_00_000
INDEX_LEVERAGE = 4
SLIPPAGE_FACTOR = 0.0005

In [217]:
# def generate_pivots(df):

#     df["Bullish Pivot"] = 0
#     df["Bearish Pivot"] = 0
#     df["MA"] = df["close"].rolling(window=28).mean()

#     # Generate signals using boolean masking
#     bull_pivot_mask = (df["low"] >= df["low"].shift(1)) & (
#         df["low"].shift(2) >= df["low"].shift(1)
#     )
#     bear_pivot_mask = (df["high"] <= df["high"].shift(1)) & (
#         df["high"].shift(2) <= df["high"].shift(1)
#     )

#     df.loc[bull_pivot_mask, "Bullish Pivot"] = 1
#     df.loc[bear_pivot_mask, "Bearish Pivot"] = 1

#     return df


# bnf_1hr = generate_pivots(bnf_1hr)
# bnf_1hr.head()

In [218]:
def generate_pivots2(df):

    df["Bullish Pivot"] = 0
    df["Bearish Pivot"] = 0
    df["MA"] = df["close"].rolling(window=16).mean()

    # Generate signals using boolean masking
    bull_pivot_mask = (df["close"] >= df["close"].shift(1)) & (
        df["close"].shift(2) >= df["close"].shift(1)
    )
    bear_pivot_mask = (df["close"] <= df["close"].shift(1)) & (
        df["close"].shift(2) <= df["close"].shift(1)
    )

    df.loc[bull_pivot_mask, "Bullish Pivot"] = 1
    df.loc[bear_pivot_mask, "Bearish Pivot"] = 1

    return df


bnf_1hr = generate_pivots2(bnf_1hr)
bnf_1hr.head()

,open,high,low,close,datetime,Bullish Pivot,Bearish Pivot,MA
0,3069,3080,3066,3068,2015-03-02 09:00:00+05:30,0,0,NaN
1,3068,3085,3053,3056,2015-03-02 13:00:00+05:30,0,0,NaN
2,3056,3140,3040,3124,2015-03-02 17:00:00+05:30,1,0,NaN
3,3124,3177,3118,3145,2015-03-02 21:00:00+05:30,0,0,NaN
4,3134,3134,3104,3112,2015-03-03 09:00:00+05:30,0,1,NaN


In [219]:
temp_df = bnf_1hr

def execute_trades(df):
    tradebook = []

    in_trade = False
    long_trade_active = False
    short_trade_active = False

    remark = ""

    signal_entry_price = 1_00_000
    signal_initial_sl = 0

    points = 0

    sl_pct = 2.5
    target_pct = 12.5

    long_points = 0
    short_points = 0

    for i in range(0, len(df)):

        points = 0

        previous_candle_open = df.iloc[i - 1]["open"]
        previous_candle_high = df.iloc[i - 1]["high"]
        previous_candle_low = df.iloc[i - 1]["low"]
        previous_candle_close = df.iloc[i - 1]["close"]
        previous_candle_time = df.iloc[i - 1]["datetime"]
        previous_candle = df.iloc[i-1]

        current_candle_open = df.iloc[i]["open"]
        current_candle_high = df.iloc[i]["high"]
        current_candle_low = df.iloc[i]["low"]
        current_candle_close = df.iloc[i]["close"]
        current_candle_time = df.iloc[i]["datetime"]
        current_candle = df.iloc[i]

        print(
            current_candle_time,
            current_candle_open,
            current_candle_high,
            current_candle_low,
            current_candle_close,
        )

        if not in_trade:

            if (df.iloc[i - 1]["Bullish Pivot"] == 1) & (
                df.iloc[i - 1]["Bearish Pivot"] == 1
            ):
                if current_candle_close >= df.iloc[i]['MA']:
                    df.at[i - 1, "Bullish Pivot"] = 0
                elif current_candle_close < df.iloc[i]['MA']:
                    df.at[i - 1, "Bearish Pivot"] = 0

            if df.iloc[i - 1]["Bullish Pivot"] == 1:
                # Long Side
                signal_time = previous_candle_time

                if current_candle_high >= previous_candle_high and (previous_candle_high < df.iloc[i]['MA']):
                    if current_candle_open > previous_candle_high:
                        # Check Entry Skip
                        print("Long Entry Skipped", current_candle_time)
                        in_trade = False
                        signal_entry_price = 1_00_000
                        signal_initial_sl = 0
                        continue
                    else:
                        # Entry Triggered
                        print("Long Entry Triggered", current_candle_time)
                        in_trade = True
                        long_trade_active = True
                        entry_time = current_candle_time
                        entry_price = previous_candle_high
                        initial_sl = entry_price * (1 - (sl_pct / 100))
                        target = entry_price * (1 + (target_pct / 100))
                        qty = PORTFOLIO * INDEX_LEVERAGE / entry_price
                        
            if df.iloc[i - 1]["Bearish Pivot"] == 1:
                # Short Side
                signal_time = previous_candle_time

                if current_candle_low <= previous_candle_low and (previous_candle_low > df.iloc[i]['MA']):
                    if current_candle_open < previous_candle_low:
                        # Check Entry Skip
                        print("Short Entry Skipped", current_candle_time)
                        in_trade = False
                        signal_entry_price = 1_00_000
                        signal_initial_sl = 0
                        continue
                    else:
                        # Entry Triggered
                        print("Short Entry Triggered", current_candle_time)
                        in_trade = True
                        short_trade_active = True
                        entry_time = current_candle_time
                        entry_price = previous_candle_low
                        initial_sl = entry_price * (1 + (sl_pct / 100))
                        target = entry_price * (1 - (target_pct / 100))
                        qty = PORTFOLIO * INDEX_LEVERAGE / entry_price

        if in_trade:
            if long_trade_active:
                if current_candle_open <= initial_sl:
                    # Gap Outside Initial SL
                    in_trade = False
                    long_trade_active = False
                    exit_price = current_candle_open
                    exit_time = current_candle_time
                    long_points = exit_price - entry_price
                    remark = "Gap Outside Initial SL"
                    print(remark, current_candle_time, exit_price)

                # elif (df.iloc[i - 1]["Bearish Pivot"] == 1) and (current_candle_low <= previous_candle_low) and (not(current_candle_open < previous_candle_low)):
                # # if (current_candle_low < previous_candle_low) and (not(current_candle_open < previous_candle_low)): # Ensuring that there was no Gap for entry
                #     # Reversal Trade
                #     exit_price = previous_candle_low
                #     exit_time = current_candle_time
                #     long_points = exit_price - entry_price
                #     remark = "Reversed"
                #     print(remark, current_candle_time, exit_price)
                #     trade = {
                #         "Trade Type": "LONG",
                #         "Signal Generated At": signal_time,
                #         "Entry Time": entry_time,
                #         "Entry Price": entry_price,
                #         "Initial SL": initial_sl,
                #         "Exit Time": exit_time,
                #         "Exit Price": exit_price,
                #         "Remarks": remark,
                #         "Points Captured": long_points,
                #         "Points w CS": long_points
                #         - (SLIPPAGE_FACTOR * (entry_price + exit_price)),
                #         "Qty": qty,
                #         "Leverage": INDEX_LEVERAGE,
                #         "PnL": qty * long_points,
                #         "PnL w CS": qty
                #         * (
                #             long_points
                #             - (SLIPPAGE_FACTOR * (entry_price + exit_price))
                #         ),
                #         "ROI%": qty * long_points * 100 / PORTFOLIO,
                #         "ROI% w CS": qty
                #         * (
                #             long_points
                #             - (SLIPPAGE_FACTOR * (entry_price + exit_price))
                #         )
                #         * 100
                #         / PORTFOLIO,
                #         "Trade Year": entry_time.year,
                #     }
                #     tradebook.append(trade)
    
                #     long_points = 0

                #     long_trade_active = False
                #     short_trade_active = True
                #     in_trade = True

                #     entry_price = previous_candle_low
                #     initial_sl = df.iloc[i - 2]["high"]
                #     signal_time = previous_candle_time
                #     entry_time = current_candle_time
                #     print(signal_time, entry_time, entry_price, initial_sl)
                #     qty = PORTFOLIO * INDEX_LEVERAGE / entry_price
                #     # continue

                #     if current_candle_high >= initial_sl:
                #         if not (current_candle_open > initial_sl): # Ensuring that there was no Gap for entry
                #             # Initial SL Hit
                #             # print(current_candle)
                #             in_trade = False
                #             long_trade_active = False
                #             exit_price = initial_sl
                #             exit_time = current_candle_time
                #             short_points = entry_price - exit_price
                #             remark = "Initial SL Hit"
                #             print(remark, current_candle_time, exit_price)

                #             trade = {
                #                 "Trade Type": "SHORT",
                #                 "Signal Generated At": signal_time,
                #                 "Entry Time": entry_time,
                #                 "Entry Price": entry_price,
                #                 "Initial SL": initial_sl,
                #                 "Exit Time": exit_time,
                #                 "Exit Price": exit_price,
                #                 "Remarks": remark,
                #                 "Points Captured": short_points,
                #                 "Points w CS": short_points
                #                 - (SLIPPAGE_FACTOR * (entry_price + exit_price)),
                #                 "Qty": qty,
                #                 "Leverage": INDEX_LEVERAGE,
                #                 "PnL": qty * short_points,
                #                 "PnL w CS": qty
                #                 * (
                #                     short_points
                #                     - (SLIPPAGE_FACTOR * (entry_price + exit_price))
                #                 ),
                #                 "ROI%": qty * short_points * 100 / PORTFOLIO,
                #                 "ROI% w CS": qty
                #                 * (
                #                     short_points
                #                     - (SLIPPAGE_FACTOR * (entry_price + exit_price))
                #                 )
                #                 * 100
                #                 / PORTFOLIO,
                #                 "Trade Year": entry_time.year,
                #             }
                #             tradebook.append(trade)
        
                #             short_points = 0
                #             short_trade_active = False
                            

                elif current_candle_low <= initial_sl:
                    # Initial SL Hit
                    in_trade = False
                    long_trade_active = False
                    exit_price = initial_sl
                    exit_time = current_candle_time
                    long_points = exit_price - entry_price
                    remark = "Initial SL Hit"
                    print(remark, current_candle_time, exit_price)

                elif current_candle_high >= target:
                    in_trade = False
                    long_trade_active = False
                    exit_price = target
                    exit_time = current_candle_time
                    long_points = exit_price - entry_price
                    remark = "Target Hit"
                    print(remark, current_candle_time, exit_price)

                if long_points:
                    trade = {
                        "Trade Type": "LONG",
                        "Signal Generated At": signal_time,
                        "Entry Time": entry_time,
                        "Entry Price": entry_price,
                        "Initial SL": initial_sl,
                        "Exit Time": exit_time,
                        "Exit Price": exit_price,
                        "Remarks": remark,
                        "Points Captured": long_points,
                        "Points w CS": long_points
                        - (SLIPPAGE_FACTOR * (entry_price + exit_price)),
                        "Qty": qty,
                        "Leverage": INDEX_LEVERAGE,
                        "PnL": qty * long_points,
                        "PnL w CS": qty
                        * (
                            long_points - (SLIPPAGE_FACTOR * (entry_price + exit_price))
                        ),
                        "ROI%": qty * long_points * 100 / PORTFOLIO,
                        "ROI% w CS": qty
                        * (long_points - (SLIPPAGE_FACTOR * (entry_price + exit_price)))
                        * 100
                        / PORTFOLIO,
                        "Trade Year": entry_time.year,
                    }
                    tradebook.append(trade)

                    long_points = 0
                    long_trade_active = False

            if short_trade_active:
                if current_candle_open >= initial_sl:
                    # Gap Outside Initial SL
                    in_trade = False
                    short_trade_active = False
                    exit_price = current_candle_open
                    exit_time = current_candle_time
                    short_points = entry_price - exit_price
                    remark = "Gap Outside Initial SL"
                    print(remark, current_candle_time, exit_price)

                # elif (df.iloc[i - 1]["Bullish Pivot"] == 1) and (current_candle_high >= previous_candle_high) and (not(current_candle_open > previous_candle_high)):
                #     # if (current_candle_high > previous_candle_high) and (not(current_candle_open > previous_candle_high)): # Ensuring that there was no Gap for entry
                #     # Reversal Tradce
                #     exit_price = previous_candle_high
                #     exit_time = current_candle_time
                #     short_points = entry_price - exit_price
                #     remark = "Reversed"
                #     print(remark, current_candle_time, exit_price)
                #     trade = {
                #         "Trade Type": "SHORT",
                #         "Signal Generated At": signal_time,
                #         "Entry Time": entry_time,
                #         "Entry Price": entry_price,
                #         "Initial SL": initial_sl,
                #         "Exit Time": exit_time,
                #         "Exit Price": exit_price,
                #         "Remarks": remark,
                #         "Points Captured": short_points,
                #         "Points w CS": short_points
                #         - (SLIPPAGE_FACTOR * (entry_price + exit_price)),
                #         "Qty": qty,
                #         "Leverage": INDEX_LEVERAGE,
                #         "PnL": qty * short_points,
                #         "PnL w CS": qty
                #         * (
                #             short_points
                #             - (SLIPPAGE_FACTOR * (entry_price + exit_price))
                #         ),
                #         "ROI%": qty * short_points * 100 / PORTFOLIO,
                #         "ROI% w CS": qty
                #         * (
                #             short_points
                #             - (SLIPPAGE_FACTOR * (entry_price + exit_price))
                #         )
                #         * 100
                #         / PORTFOLIO,
                #         "Trade Year": entry_time.year,
                #     }
                #     tradebook.append(trade)

                #     short_points = 0

                #     long_trade_active = True
                #     short_trade_active = False
                #     in_trade = True

                #     entry_price = previous_candle_high
                #     initial_sl = df.iloc[i - 2]["low"]
                #     signal_time = previous_candle_time
                #     entry_time = current_candle_time
                #     qty = PORTFOLIO * INDEX_LEVERAGE / entry_price
                #     print(signal_time, entry_time, entry_price, initial_sl)
                #     # continue

                #     if current_candle_low <= initial_sl:
                #         if not (current_candle_open < initial_sl): # Ensuring that there was no Gap for entry
                #             # Initial SL Hit
                #             # print(current_candle)
                #             in_trade = False
                #             short_trade_active = False
                #             exit_price = initial_sl
                #             exit_time = current_candle_time
                #             long_points = exit_price - entry_price
                #             remark = "Initial SL Hit"
                #             print(remark, current_candle_time, exit_price)

                #             trade = {
                #                 "Trade Type": "LONG",
                #                 "Signal Generated At": signal_time,
                #                 "Entry Time": entry_time,
                #                 "Entry Price": entry_price,
                #                 "Initial SL": initial_sl,
                #                 "Exit Time": exit_time,
                #                 "Exit Price": exit_price,
                #                 "Remarks": remark,
                #                 "Points Captured": long_points,
                #                 "Points w CS": long_points
                #                 - (SLIPPAGE_FACTOR * (entry_price + exit_price)),
                #                 "Qty": qty,
                #                 "Leverage": INDEX_LEVERAGE,
                #                 "PnL": qty * long_points,
                #                 "PnL w CS": qty
                #                 * (
                #                     long_points - (SLIPPAGE_FACTOR * (entry_price + exit_price))
                #                 ),
                #                 "ROI%": qty * long_points * 100 / PORTFOLIO,
                #                 "ROI% w CS": qty
                #                 * (long_points - (SLIPPAGE_FACTOR * (entry_price + exit_price)))
                #                 * 100
                #                 / PORTFOLIO,
                #                 "Trade Year": entry_time.year,
                #             }
                #             tradebook.append(trade)
        
                #             long_points = 0
                #             long_trade_active = False

                elif current_candle_high >= initial_sl:
                    # Initial SL Hit
                    in_trade = False
                    short_trade_active = False
                    exit_price = initial_sl
                    exit_time = current_candle_time
                    short_points = entry_price - exit_price
                    remark = "Initial SL Hit"
                    print(remark, current_candle_time, exit_price)

                elif current_candle_low <= target:
                    in_trade = False
                    short_trade_active = False
                    exit_price = target
                    exit_time = current_candle_time
                    short_points = entry_price - exit_price
                    remark = "Target Hit"
                    print(remark, current_candle_time, exit_price)

                if short_points:
                    trade = {
                        "Trade Type": "SHORT",
                        "Signal Generated At": signal_time,
                        "Entry Time": entry_time,
                        "Entry Price": entry_price,
                        "Initial SL": initial_sl,
                        "Exit Time": exit_time,
                        "Exit Price": exit_price,
                        "Remarks": remark,
                        "Points Captured": short_points,
                        "Points w CS": short_points
                        - (SLIPPAGE_FACTOR * (entry_price + exit_price)),
                        "Qty": qty,
                        "Leverage": INDEX_LEVERAGE,
                        "PnL": qty * short_points,
                        "PnL w CS": qty
                        * (
                            short_points
                            - (SLIPPAGE_FACTOR * (entry_price + exit_price))
                        ),
                        "ROI%": qty * short_points * 100 / PORTFOLIO,
                        "ROI% w CS": qty
                        * (
                            short_points
                            - (SLIPPAGE_FACTOR * (entry_price + exit_price))
                        )
                        * 100
                        / PORTFOLIO,
                        "Trade Year": entry_time.year,
                    }
                    tradebook.append(trade)

                    short_points = 0
                    short_trade_active = False

    trade_book_df = pd.DataFrame(tradebook)
    return trade_book_df


tb = execute_trades(bnf_1hr)

2015-03-02 09:00:00+05:30 3069 3080 3066 3068
2015-03-02 13:00:00+05:30 3068 3085 3053 3056
2015-03-02 17:00:00+05:30 3056 3140 3040 3124
2015-03-02 21:00:00+05:30 3124 3177 3118 3145
2015-03-03 09:00:00+05:30 3134 3134 3104 3112
2015-03-03 13:00:00+05:30 3113 3147 3108 3133
2015-03-03 17:00:00+05:30 3133 3155 3083 3115
2015-03-03 21:00:00+05:30 3114 3151 3102 3137
2015-03-04 09:00:00+05:30 3148 3150 3139 3148
2015-03-04 13:00:00+05:30 3148 3184 3141 3170
2015-03-04 17:00:00+05:30 3172 3200 3157 3168
2015-03-04 21:00:00+05:30 3147 3180 3112 3177
2015-03-05 09:00:00+05:30 3189 3245 3189 3236
2015-03-05 13:00:00+05:30 3237 3279 3225 3256
2015-03-05 17:00:00+05:30 3255 3258 3185 3215
2015-03-05 21:00:00+05:30 3215 3248 3200 3210
2015-03-06 17:00:00+05:30 3209 3209 3143 3178
2015-03-06 21:00:00+05:30 3178 3178 3121 3121
2015-03-09 09:00:00+05:30 3120 3132 3107 3125
2015-03-09 13:00:00+05:30 3125 3138 3109 3125
Long Entry Triggered 2015-03-09 13:00:00+05:30
2015-03-09 17:00:00+05:30 3125 31

In [220]:
tb

,Trade Type,Signal Generated At,Entry Time,Entry Price,Initial SL,Exit Time,Exit Price,Remarks,Points Captured,Points w CS,Qty,Leverage,PnL,PnL w CS,ROI%,ROI% w CS,Trade Year
0,LONG,2015-03-09 09:00:00+05:30,2015-03-09 13:00:00+05:30,3132,3053.7000,2015-03-10 21:00:00+05:30,3053.7000,Initial SL Hit,-78.3000,-81.3929,6385.6960,4,-500000.0000,-519750.0000,-10.0000,-10.3950,2015
1,LONG,2015-03-12 09:00:00+05:30,2015-03-12 13:00:00+05:30,3047,2970.8250,2015-03-12 21:00:00+05:30,2970.8250,Initial SL Hit,-76.1750,-79.1839,6563.8333,4,-500000.0000,-519750.0000,-10.0000,-10.3950,2015
2,LONG,2015-03-13 09:00:00+05:30,2015-03-13 13:00:00+05:30,2974,2899.6500,2015-03-13 17:00:00+05:30,2899.6500,Initial SL Hit,-74.3500,-77.2868,6724.9496,4,-500000.0000,-519750.0000,-10.0000,-10.3950,2015
3,LONG,2015-03-17 17:00:00+05:30,2015-03-17 21:00:00+05:30,2756,2687.1000,2015-03-18 09:00:00+05:30,2687.1000,Initial SL Hit,-68.9000,-71.6216,7256.8940,4,-500000.0000,-519750.0000,-10.0000,-10.3950,2015
4,SHORT,2015-03-24 17:00:00+05:30,2015-03-24 21:00:00+05:30,2981,3055.5250,2015-03-25 21:00:00+05:30,3055.5250,Initial SL Hit,-74.5250,-77.5433,6709.1580,4,-500000.0000,-520250.0000,-10.0000,-10.4050,2015
5,SHORT,2015-03-26 13:00:00+05:30,2015-03-26 17:00:00+05:30,3223,3303.5750,2015-04-07 17:00:00+05:30,3303.5750,Initial SL Hit,-80.5750,-83.8383,6205.3987,4,-500000.0000,-520250.0000,-10.0000,-10.4050,2015
6,SHORT,2015-04-08 09:00:00+05:30,2015-04-08 13:00:00+05:30,3303,3385.5750,2015-04-15 09:00:00+05:30,3385.5750,Initial SL Hit,-82.5750,-85.9193,6055.1014,4,-500000.0000,-520250.0000,-10.0000,-10.4050,2015
7,SHORT,2015-04-15 13:00:00+05:30,2015-04-15 17:00:00+05:30,3358,3441.9500,2015-04-15 17:00:00+05:30,3441.9500,Initial SL Hit,-83.9500,-87.3500,5955.9261,4,-500000.0000,-520250.0000,-10.0000,-10.4050,2015
8,SHORT,2015-04-16 13:00:00+05:30,2015-04-16 17:00:00+05:30,3448,3534.2000,2015-04-16 21:00:00+05:30,3534.2000,Initial SL Hit,-86.2000,-89.6911,5800.4640,4,-500000.0000,-520250.0000,-10.0000,-10.4050,2015
9,SHORT,2015-04-17 09:00:00+05:30,2015-04-17 13:00:00+05:30,3500,3587.5000,2015-04-21 09:00:00+05:30,3651.0000,Gap Outside Initial SL,-151.0000,-154.5755,5714.2857,4,-862857.1429,-883288.5714,-17.2571,-17.6658,2015


In [221]:
tb["ROI% w CS"].sum()

-744.0477711412101

In [222]:
stats_df8 = pd.DataFrame(
    index=range(2015, 2025),
    columns=[
        "Total ROI",
        "Total Trades",
        "Win Rate",
        "Avg Profit% per Trade",
        "Avg Loss% per Trade",
        "Max Drawdown",
        "ROI/DD Ratio",
    ],
)
combined_df_sorted = tb
# Iterate over each year
for year in range(2015, 2025):
    # Filter trades for the current year
    year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

    # Calculate total ROI
    total_roi = year_trades["ROI% w CS"].sum()

    # Calculate total number of trades
    total_trades = len(year_trades)

    # Calculate win rate
    win_rate = (year_trades["ROI% w CS"] > 0).mean() * 100

    # Calculate average profit per trade
    avg_profit = year_trades[year_trades["ROI% w CS"] > 0]["ROI% w CS"].mean()

    # Calculate average loss per trade
    avg_loss = year_trades[year_trades["ROI% w CS"] < 0]["ROI% w CS"].mean()

    # Calculate maximum drawdown
    max_drawdown = (
        year_trades["ROI% w CS"].cumsum() - year_trades["ROI% w CS"].cumsum().cummax()
    ).min()

    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)

    # Store the statistics in the DataFrame
    stats_df8.loc[year] = [
        total_roi,
        total_trades,
        win_rate,
        avg_profit,
        avg_loss,
        max_drawdown,
        roi_dd_ratio,
    ]

# Calculate overall statistics
overall_total_roi = stats_df8["Total ROI"].sum()
overall_total_trades = stats_df8["Total Trades"].sum()
overall_win_rate = (combined_df_sorted["ROI% w CS"] > 0).mean() * 100
overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI% w CS"] > 0][
    "ROI% w CS"
].mean()
overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI% w CS"] < 0][
    "ROI% w CS"
].mean()
overall_max_drawdown = (
    combined_df_sorted["ROI% w CS"].cumsum()
    - combined_df_sorted["ROI% w CS"].cumsum().cummax()
).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df8.loc["Overall"] = [
    overall_total_roi,
    overall_total_trades,
    overall_win_rate,
    overall_avg_profit,
    overall_avg_loss,
    overall_max_drawdown,
    overall_roi_dd_ratio,
]
stats_df8

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2015,-255.9358,47,8.5106,49.6250,-10.5683,-290.4158,-0.8813
2016,-87.4427,47,14.8936,49.5964,-10.8654,-131.9479,-0.6627
2017,12.3658,16,18.7500,49.5917,-10.4930,-84.4342,0.1465
2018,-13.5824,30,16.6667,49.5850,-10.4603,-145.5400,-0.0933
2019,-80.0150,25,12.0000,49.6083,-10.4018,-148.0200,-0.5406
2020,-335.7144,42,7.1429,49.5917,-12.4228,-390.8844,-0.8589
2021,68.2867,28,21.4286,49.6000,-10.4233,-83.2300,0.8205
2022,40.8500,48,18.7500,49.6083,-10.4006,-104.0100,0.3928
2023,21.5350,21,19.0476,49.5875,-10.4009,-145.6100,0.1479
2024,-114.3950,11,0.0000,NaN,-10.3995,-103.9900,-1.1001


In [119]:
# tb.to_csv(f"BNF Rolling Pivots {INDEX_LEVERAGE}x Lev.csv")